In [8]:
# ============================================================
# שלב 1 ו-2: Data Exploration & Cleaning
# Crime Analysis Project - Chicago Crime Data
# ============================================================

import pandas as pd
import numpy as np
import os

print("=" * 60)
print("שלב 1: טעינת הנתונים")
print("=" * 60)

# בדיקה שהקובץ קיים
filename = "Chicago_Crimes_2012_to_2017.csv"
if os.path.exists(filename):
    print(f"✓ הקובץ נמצא: {filename}")
    print("טוען נתונים... (זה יכול לקחת 1-2 דקות)")
    
    # טעינת הנתונים
    df = pd.read_csv(filename)
    
    print(f"\n🎉 הנתונים נטענו בהצלחה!")
    print(f"מספר שורות: {len(df):,}")
    print(f"מספר עמודות: {len(df.columns)}")
else:
    print(f"❌ הקובץ לא נמצא: {filename}")
    print("אנא ודא שהקובץ נמצא בתיקיית הפרויקט!")

print("\n" + "=" * 60)
print("שלב 2: Data Cleaning")
print("=" * 60)

# המרת התאריך
print("\n1. המרת עמודת התאריך...")
df['Date'] = pd.to_datetime(df['Date'], format='%m/%d/%Y %I:%M:%S %p')
print("✓ התאריכים הומרו בהצלחה!")

# יצירת עמודות נוספות
print("\n2. יצירת עמודות זמן...")
df['Month'] = df['Date'].dt.month
df['Day_of_Week'] = df['Date'].dt.dayofweek
df['Day_Name'] = df['Date'].dt.day_name()
df['Hour'] = df['Date'].dt.hour
df['Month_Name'] = df['Date'].dt.month_name()
print("✓ נוצרו 5 עמודות חדשות!")

# טיפול בערכים חסרים
print("\n3. טיפול בערכים חסרים...")
before = len(df)
df_clean = df.dropna(subset=['Latitude', 'Longitude']).copy()
df_clean['Location Description'] = df_clean['Location Description'].fillna('UNKNOWN')
after = len(df_clean)
removed = before - after
print(f"✓ נמחקו {removed:,} שורות ללא GPS ({(removed/before)*100:.2f}%)")
print(f"✓ נשארו {after:,} שורות נקיות")

# שמירת הדאטה הנקי
print("\n4. שמירת הדאטה הנקי...")
output_file = "Chicago_Crimes_Clean.csv"
df_clean.to_csv(output_file, index=False)
print(f"✓ הקובץ נשמר: {output_file}")

# סטטיסטיקות סופיות
print("\n" + "=" * 60)
print("סיכום")
print("=" * 60)
print(f"✓ דאטה נקי: {len(df_clean):,} שורות, {len(df_clean.columns)} עמודות")
print(f"✓ טווח שנים: {df_clean['Year'].min()}-{df_clean['Year'].max()}")
print(f"✓ סוגי פשיעה: {df_clean['Primary Type'].nunique()}")
print(f"✓ אחוז מעצרים: {(df_clean['Arrest'].sum() / len(df_clean)) * 100:.2f}%")

print("\n🎯 Top 5 סוגי פשיעה:")
print(df_clean['Primary Type'].value_counts().head(5))

print("\n✅ שלב 1 ו-2 הושלמו בהצלחה!")

שלב 1: טעינת הנתונים
✓ הקובץ נמצא: Chicago_Crimes_2012_to_2017.csv
טוען נתונים... (זה יכול לקחת 1-2 דקות)

🎉 הנתונים נטענו בהצלחה!
מספר שורות: 1,456,714
מספר עמודות: 23

שלב 2: Data Cleaning

1. המרת עמודת התאריך...
✓ התאריכים הומרו בהצלחה!

2. יצירת עמודות זמן...
✓ נוצרו 5 עמודות חדשות!

3. טיפול בערכים חסרים...
✓ נמחקו 37,083 שורות ללא GPS (2.55%)
✓ נשארו 1,419,631 שורות נקיות

4. שמירת הדאטה הנקי...
✓ הקובץ נשמר: Chicago_Crimes_Clean.csv

סיכום
✓ דאטה נקי: 1,419,631 שורות, 28 עמודות
✓ טווח שנים: 2012-2017
✓ סוגי פשיעה: 33
✓ אחוז מעצרים: 26.14%

🎯 Top 5 סוגי פשיעה:
Primary Type
THEFT              322433
BATTERY            258945
CRIMINAL DAMAGE    152817
NARCOTICS          131184
ASSAULT             89512
Name: count, dtype: int64

✅ שלב 1 ו-2 הושלמו בהצלחה!


In [10]:
# בדיקה מהירה שהדאטה עובד
print("=== בדיקה סופית ===\n")
print("5 שורות ראשונות:")
df_clean[['Date', 'Primary Type', 'Location Description', 'Arrest', 'Hour', 'Day_Name']].head()

=== בדיקה סופית ===

5 שורות ראשונות:


,Date,Primary Type,Location Description,Arrest,Hour,Day_Name
0,2016-05-03 23:40:00,BATTERY,APARTMENT,True,23,Tuesday
1,2016-05-03 21:40:00,BATTERY,RESIDENCE,False,21,Tuesday
2,2016-05-03 23:31:00,PUBLIC PEACE VIOLATION,STREET,False,23,Tuesday
3,2016-05-03 22:10:00,BATTERY,SIDEWALK,False,22,Tuesday
4,2016-05-03 22:00:00,THEFT,RESIDENCE,False,22,Tuesday


In [11]:
# ============================================================
# שלב 3: טעינת נתונים ל-PostgreSQL
# ============================================================

import psycopg2
from sqlalchemy import create_engine

print("=" * 60)
print("שלב 3: טעינת נתונים ל-PostgreSQL")
print("=" * 60)

# יצירת חיבור למסד הנתונים
print("\n1. מתחבר למסד הנתונים...")

# הגדרת פרטי החיבור
db_params = {
    'host': 'localhost',
    'port': 5433,
    'database': 'crime_analysis',
    'user': 'postgres',
    'password': '12345678'
}

# יצירת engine ל-SQLAlchemy (זה עוזר להעלות DataFrame ישירות)
engine = create_engine(f"postgresql://{db_params['user']}:{db_params['password']}@{db_params['host']}:{db_params['port']}/{db_params['database']}")

print("✓ חיבור הוקם בהצלחה!")

# טעינת הנתונים למסד הנתונים
print("\n2. מעלה נתונים למסד הנתונים...")
print("   (זה ייקח כמה דקות - אנחנו מעלים 1.4 מיליון שורות!)")

# נטען רק דגימה קטנה תחילה (10,000 שורות) לבדיקה
sample_size = 10000
df_sample = df_clean.head(sample_size)

# מיפוי שמות עמודות ל-SQL
df_sample_renamed = df_sample.rename(columns={
    'Unnamed: 0': 'original_id',
    'ID': 'chicago_id',
    'Case Number': 'case_number',
    'Date': 'date_occurred',
    'Block': 'block',
    'IUCR': 'iucr',
    'Primary Type': 'primary_type',
    'Description': 'description',
    'Location Description': 'location_description',
    'Arrest': 'arrest',
    'Domestic': 'domestic',
    'Beat': 'beat',
    'District': 'district',
    'Ward': 'ward',
    'Community Area': 'community_area',
    'FBI Code': 'fbi_code',
    'X Coordinate': 'x_coordinate',
    'Y Coordinate': 'y_coordinate',
    'Year': 'year',
    'Latitude': 'latitude',
    'Longitude': 'longitude',
    'Month': 'month',
    'Day_of_Week': 'day_of_week',
    'Day_Name': 'day_name',
    'Hour': 'hour',
    'Month_Name': 'month_name'
})

# בחירת העמודות הרלוונטיות
columns_to_upload = [
    'case_number', 'date_occurred', 'block', 'iucr', 'primary_type',
    'description', 'location_description', 'arrest', 'domestic',
    'beat', 'district', 'ward', 'community_area', 'fbi_code',
    'x_coordinate', 'y_coordinate', 'year', 'latitude', 'longitude',
    'month', 'day_of_week', 'day_name', 'hour', 'month_name'
]

df_to_upload = df_sample_renamed[columns_to_upload]

# העלאה למסד הנתונים
df_to_upload.to_sql('crimes', engine, if_exists='append', index=False)

print(f"✓ הועלו {len(df_to_upload):,} שורות למסד הנתונים!")

print("\n" + "=" * 60)
print("✅ שלב 3 הושלם בהצלחה!")
print("=" * 60)

שלב 3: טעינת נתונים ל-PostgreSQL

1. מתחבר למסד הנתונים...
✓ חיבור הוקם בהצלחה!

2. מעלה נתונים למסד הנתונים...
   (זה ייקח כמה דקות - אנחנו מעלים 1.4 מיליון שורות!)
✓ הועלו 10,000 שורות למסד הנתונים!

✅ שלב 3 הושלם בהצלחה!


In [12]:
# בדיקה שהנתונים נטענו בהצלחה
print("=== בדיקת נתונים במסד הנתונים ===\n")

# חיבור למסד הנתונים
conn = psycopg2.connect(**db_params)
cursor = conn.cursor()

# שאילתת SQL פשוטה - כמה שורות יש?
cursor.execute("SELECT COUNT(*) FROM crimes;")
count = cursor.fetchone()[0]
print(f"✓ מספר שורות במסד הנתונים: {count:,}")

# 5 השורות הראשונות
cursor.execute("SELECT primary_type, date_occurred, location_description, arrest FROM crimes LIMIT 5;")
rows = cursor.fetchall()

print("\n5 שורות ראשונות:")
print("-" * 80)
for row in rows:
    print(f"סוג: {row[0]:<30} תאריך: {row[1]}  מיקום: {row[2]:<20} מעצר: {row[3]}")

# סגירת החיבור
cursor.close()
conn.close()

print("\n✅ הנתונים במסד הנתונים!")

=== בדיקת נתונים במסד הנתונים ===

✓ מספר שורות במסד הנתונים: 30,000

5 שורות ראשונות:
--------------------------------------------------------------------------------
סוג: BATTERY                        תאריך: 2016-05-03 23:40:00  מיקום: APARTMENT            מעצר: True
סוג: BATTERY                        תאריך: 2016-05-03 21:40:00  מיקום: RESIDENCE            מעצר: False
סוג: PUBLIC PEACE VIOLATION         תאריך: 2016-05-03 23:31:00  מיקום: STREET               מעצר: False
סוג: BATTERY                        תאריך: 2016-05-03 22:10:00  מיקום: SIDEWALK             מעצר: False
סוג: THEFT                          תאריך: 2016-05-03 22:00:00  מיקום: RESIDENCE            מעצר: False

✅ הנתונים במסד הנתונים!


In [7]:
# ============================================================
# שלב 4: Basic SQL Queries - שאילתות בסיסיות
# ============================================================

import psycopg2
import pandas as pd

print("=" * 60)
print("שלב 4: Basic SQL Queries")
print("=" * 60)

# חיבור למסד הנתונים
conn = psycopg2.connect(**db_params)

print("\n1️⃣ שאילתה 1: כמה אירועי פשיעה יש במסד הנתונים?")
print("-" * 60)

query1 = "SELECT COUNT(*) as total_crimes FROM crimes;"
result1 = pd.read_sql(query1, conn)
print(f"✓ סך הכל אירועים: {result1['total_crimes'][0]:,}")

print("\n2️⃣ שאילתה 2: מהם 10 סוגי הפשיעה הנפוצים ביותר?")
print("-" * 60)

query2 = """
SELECT primary_type, COUNT(*) as count
FROM crimes
GROUP BY primary_type
ORDER BY count DESC
LIMIT 10;
"""
result2 = pd.read_sql(query2, conn)
print(result2.to_string(index=False))

print("\n3️⃣ שאילתה 3: כמה אירועים יש בכל שנה?")
print("-" * 60)

query3 = """
SELECT year, COUNT(*) as count
FROM crimes
GROUP BY year
ORDER BY year;
"""
result3 = pd.read_sql(query3, conn)
print(result3.to_string(index=False))

print("\n4️⃣ שאילתה 4: מה אחוז המעצרים?")
print("-" * 60)

query4 = """
SELECT 
    COUNT(*) as total_crimes,
    SUM(CASE WHEN arrest = TRUE THEN 1 ELSE 0 END) as arrests,
    ROUND(100.0 * SUM(CASE WHEN arrest = TRUE THEN 1 ELSE 0 END) / COUNT(*), 2) as arrest_percentage
FROM crimes;
"""
result4 = pd.read_sql(query4, conn)
print(f"✓ סך הכל אירועים: {result4['total_crimes'][0]:,}")
print(f"✓ מעצרים: {result4['arrests'][0]:,}")
print(f"✓ אחוז מעצרים: {result4['arrest_percentage'][0]}%")

# סגירת החיבור
conn.close()

print("\n" + "=" * 60)
print("✅ שלב 4 הושלם בהצלחה!")
print("=" * 60)

שלב 4: Basic SQL Queries

1️⃣ שאילתה 1: כמה אירועי פשיעה יש במסד הנתונים?
------------------------------------------------------------
✓ סך הכל אירועים: 20,000

2️⃣ שאילתה 2: מהם 10 סוגי הפשיעה הנפוצים ביותר?
------------------------------------------------------------
       primary_type  count
              THEFT   4132
            BATTERY   3946
    CRIMINAL DAMAGE   2290
          NARCOTICS   1790
            ASSAULT   1472
      OTHER OFFENSE   1180
 DECEPTIVE PRACTICE   1080
           BURGLARY    988
            ROBBERY    808
MOTOR VEHICLE THEFT    716

3️⃣ שאילתה 3: כמה אירועים יש בכל שנה?
------------------------------------------------------------
 year  count
 2012      2
 2013     10
 2014     10
 2015   1126
 2016  18852

4️⃣ שאילתה 4: מה אחוז המעצרים?
------------------------------------------------------------
✓ סך הכל אירועים: 20,000
✓ מעצרים: 4,836
✓ אחוז מעצרים: 24.18%

✅ שלב 4 הושלם בהצלחה!


C:\Users\shiri\AppData\Local\Temp\ipykernel_11532\4252715789.py:19: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  result1 = pd.read_sql(query1, conn)
C:\Users\shiri\AppData\Local\Temp\ipykernel_11532\4252715789.py:32: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  result2 = pd.read_sql(query2, conn)
C:\Users\shiri\AppData\Local\Temp\ipykernel_11532\4252715789.py:44: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  result3 = pd.read_sql(query3, conn)
C:\Users\shiri\AppData\Local\Temp\ipykernel_11532\4252715789.py:57: UserWarning

In [13]:
# ============================================================
# שאילתות מתקדמות יותר
# ============================================================

print("=" * 60)
print("שאילתות מתקדמות")
print("=" * 60)

conn = psycopg2.connect(**db_params)

print("\n5️⃣ שאילתה 5: באיזה יום בשבוע יש הכי הרבה פשיעה?")
print("-" * 60)

query5 = """
SELECT day_name, COUNT(*) as count
FROM crimes
GROUP BY day_name
ORDER BY count DESC;
"""
result5 = pd.read_sql(query5, conn)
print(result5.to_string(index=False))

print("\n6️⃣ שאילתה 6: באיזו שעה ביום יש הכי הרבה פשיעה?")
print("-" * 60)

query6 = """
SELECT hour, COUNT(*) as count
FROM crimes
GROUP BY hour
ORDER BY count DESC
LIMIT 10;
"""
result6 = pd.read_sql(query6, conn)
print(result6.to_string(index=False))

print("\n7️⃣ שאילתה 7: מהם 5 המיקומים הכי מסוכנים?")
print("-" * 60)

query7 = """
SELECT location_description, COUNT(*) as count
FROM crimes
GROUP BY location_description
ORDER BY count DESC
LIMIT 5;
"""
result7 = pd.read_sql(query7, conn)
print(result7.to_string(index=False))

print("\n8️⃣ שאילתה 8: איזה סוגי פשיעה מסתיימים במעצר הכי הרבה?")
print("-" * 60)

query8 = """
SELECT 
    primary_type,
    COUNT(*) as total,
    SUM(CASE WHEN arrest = TRUE THEN 1 ELSE 0 END) as arrests,
    ROUND(100.0 * SUM(CASE WHEN arrest = TRUE THEN 1 ELSE 0 END) / COUNT(*), 2) as arrest_rate
FROM crimes
GROUP BY primary_type
HAVING COUNT(*) > 100
ORDER BY arrest_rate DESC
LIMIT 10;
"""
result8 = pd.read_sql(query8, conn)
print(result8.to_string(index=False))

conn.close()

print("\n" + "=" * 60)
print("✅ שאילתות מתקדמות הושלמו!")
print("=" * 60)

שאילתות מתקדמות

5️⃣ שאילתה 5: באיזה יום בשבוע יש הכי הרבה פשיעה?
------------------------------------------------------------
 day_name  count
   Friday   6324
Wednesday   4518
 Saturday   4428
   Sunday   4410
 Thursday   4389
  Tuesday   3198
   Monday   2733

6️⃣ שאילתה 6: באיזו שעה ביום יש הכי הרבה פשיעה?
------------------------------------------------------------
 hour  count
   19   1806
   16   1797
   18   1797
   21   1686
   17   1653
   22   1620
   15   1608
   11   1572
   20   1530
   14   1500

7️⃣ שאילתה 7: מהם 5 המיקומים הכי מסוכנים?
------------------------------------------------------------
location_description  count
              STREET   6819
           RESIDENCE   4749
           APARTMENT   3774
            SIDEWALK   2889
               OTHER   1077

8️⃣ שאילתה 8: איזה סוגי פשיעה מסתיימים במעצר הכי הרבה?
------------------------------------------------------------
                    primary_type  total  arrests  arrest_rate
                       NARCOTICS 

C:\Users\shiri\AppData\Local\Temp\ipykernel_11532\837915789.py:20: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  result5 = pd.read_sql(query5, conn)
C:\Users\shiri\AppData\Local\Temp\ipykernel_11532\837915789.py:33: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  result6 = pd.read_sql(query6, conn)
C:\Users\shiri\AppData\Local\Temp\ipykernel_11532\837915789.py:46: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  result7 = pd.read_sql(query7, conn)
C:\Users\shiri\AppData\Local\Temp\ipykernel_11532\837915789.py:64: UserWarning: pa